In [1]:
import os 
os.chdir("../")

In [2]:
import pandas as pd
df = pd.read_csv("data/dataset.csv")

C:\Users\windows 10\AppData\Local\Temp\ipykernel_6576\685072172.py:2: DtypeWarning: Columns (33,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/dataset.csv")


### Handle missing vlaue

In [3]:
from scripts.preprocessing import replace_missing_with_mean, replace_missing_with_mode

df = df.drop(columns=['NumberOfVehiclesInFleet','CrossBorder' ])
numerical_cols = df.select_dtypes(include=['int64', 'float64'])
categorical_cols = df.select_dtypes(include='object')

df = replace_missing_with_mode(df,categorical_cols)
df = replace_missing_with_mean(df,numerical_cols.columns)


### Data preparation

In [15]:
from scripts.data_preparation import prepar_data


X_train, X_test, y_train, y_test = prepar_data(df)